In [1]:
import earthaccess
import xarray as xr

In [43]:
# Install the PR with 
# pip install git+https://github.com/zarr-developers/VirtualiZarr.git
from virtualizarr import open_virtual_dataset

In [2]:
earthaccess.login()

## Try with the *MUR25-JPL-L4-GLOB-v04.2* collection

In [3]:
granule_metadata = earthaccess.search_data(short_name="MUR25-JPL-L4-GLOB-v04.2", count=100, cloud_hosted=True)

Granules found: 8039


**Only one link in the data links**

In [19]:
granule_metadata[0].data_links()

['https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/MUR25-JPL-L4-GLOB-v04.2/20020901090000-JPL-L4_GHRSST-SSTfnd-MUR25-GLOB-v02.0-fv04.2.nc']

**dmrpp file paths can be obtained by appending "dmrpp" to the granule paths**

In [27]:
paths_dmrpp = [gm.data_links()[0]+".dmrpp" for gm in granule_metadata]
paths_dmrpp[0]

'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/MUR25-JPL-L4-GLOB-v04.2/20020901090000-JPL-L4_GHRSST-SSTfnd-MUR25-GLOB-v02.0-fv04.2.nc.dmrpp'

In [29]:
%%time
virtual_datasets = [
    open_virtual_dataset(p, indexes={}, filetype="dmrpp", reader_options={"storage_options": fs.storage_options})
    for p in paths_dmrpp
]

CPU times: user 2.92 s, sys: 166 ms, total: 3.08 s
Wall time: 2min 22s


In [30]:
virtual_datasets[0]

<xarray.Dataset> Size: 10MB
Dimensions:           (time: 1, lat: 720, lon: 1440)
Coordinates:
    time              (time) int32 4B ManifestArray<shape=(1,), dtype=int32, ...
    lat               (lat) float32 3kB ManifestArray<shape=(720,), dtype=flo...
    lon               (lon) float32 6kB ManifestArray<shape=(1440,), dtype=fl...
Data variables:
    analysed_sst      (time, lat, lon) int16 2MB ManifestArray<shape=(1, 720,...
    analysis_error    (time, lat, lon) int16 2MB ManifestArray<shape=(1, 720,...
    mask              (time, lat, lon) int16 2MB ManifestArray<shape=(1, 720,...
    sea_ice_fraction  (time, lat, lon) int16 2MB ManifestArray<shape=(1, 720,...
    sst_anomaly       (time, lat, lon) int16 2MB ManifestArray<shape=(1, 720,...

In [6]:
%%time
data_nc = xr.open_mfdataset(earthaccess.open(granule_metadata[:100]))

Opening 100 granules, approx size: 0.18 GB


QUEUEING TASKS | :   0%|          | 0/100 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/100 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/100 [00:00<?, ?it/s]

CPU times: user 7.52 s, sys: 1.55 s, total: 9.07 s
Wall time: 2min 59s


In [5]:
data_nc

<xarray.Dataset>
Dimensions:           (time: 10, lat: 720, lon: 1440)
Coordinates:
  * time              (time) datetime64[ns] 2002-09-01T09:00:00 ... 2002-09-1...
  * lat               (lat) float32 -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
  * lon               (lon) float32 -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
Data variables:
    analysed_sst      (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    analysis_error    (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    mask              (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    sst_anomaly       (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
Attributes: (12/54)
    Conventions:                CF-1.7, ACDD-1.3
    title:                      Daily 0.25-degree MUR SST, Final product
    summary:                    A low-resolution version of the MUR SST analy...
    keywords:                   Oceans > Ocean Temperature > Sea Surface Temp...
    keywords_vocabulary:        NASA Global Change Master Directory (GCMD) Sc...
    standard_name_vocabulary:   NetCDF Climate and Forecast (CF) Metadata Con...
    ...                         ...
    publisher_name:             GHRSST Project Office
    publisher_url:              https://www.ghrsst.org
    publisher_email:            gpc@ghrsst.org
    file_quality_level:         3
    metadata_link:              http://podaac.jpl.nasa.gov/ws/metadata/datase...
    acknowledgment:             Please acknowledge the use of these data with...